In [19]:
import imaplib
import email
from email.header import decode_header
import os
from getpass import getpass
import pandas as pd
import re

# Datos del usuario
username = "csoto981217@gmail.com"
password = "xtwm pmeh etfv dfda"

# Crear conexión
imap = imaplib.IMAP4_SSL("imap.gmail.com")
# Iniciar sesión
imap.login(username, password)

status, mensajes = imap.select("INBOX")
# mensajes a recibir
N = 5
# cantidad total de correos
mensajes = int(mensajes[0])

# Lista para almacenar los datos de los correos
data = []

for i in range(mensajes, mensajes - N, -1):
    try:
        res, mensaje = imap.fetch(str(i), "(RFC822)")
    except:
        break
    for respuesta in mensaje:
        if isinstance(respuesta, tuple):
            # Obtener el contenido
            mensaje = email.message_from_bytes(respuesta[1])
            # decodificar el contenido
            subject, encoding = decode_header(mensaje["Subject"])[0]
            if isinstance(subject, bytes):
                # convertir a string
                subject = subject.decode(encoding if encoding else 'utf-8', errors='replace')
            # de donde viene el correo
            from_ = mensaje.get("From")
            
            # Inicializar el cuerpo del correo
            body = ""

            if mensaje.is_multipart():
                # Recorrer las partes del correo
                for part in mensaje.walk():
                    # Extraer el contenido
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # el cuerpo del correo
                        part_body = part.get_payload(decode=True).decode('utf-8', errors='replace')
                    except:
                        part_body = ""
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # Concatenar el cuerpo del correo
                        body += part_body
            else:
                # contenido del mensaje
                content_type = mensaje.get_content_type()
                # cuerpo del mensaje
                body = mensaje.get_payload(decode=True).decode('utf-8', errors='replace')
            
            # Añadir los datos del correo a la lista
            data.append([subject, from_, body])
            print("Mensaje obtenido con éxito")

# Cerrar la conexión
imap.close()
imap.logout()

# Crear el DataFrame
df = pd.DataFrame(data, columns=["Subject", "From", "Mensaje"])

# Mostrar el DataFrame
print(df)
def limpiar_mensaje(mensaje):
    # Eliminar espacios en blanco al inicio y al final
    mensaje = mensaje.strip()
    # Eliminar caracteres especiales usando expresiones regulares
    mensaje = re.sub(r'[^\w\s]', '', mensaje)
    # Eliminar múltiples espacios y convertirlos a un solo espacio
    mensaje = re.sub(r'\s+', ' ', mensaje)
    # Convertir a minúsculas
    mensaje = mensaje.lower()
    return mensaje

# Aplicar la función de limpieza a la columna 'mensaje'
df['Mensaje'] = df['Mensaje'].apply(limpiar_mensaje)

#xtwm pmeh etfv dfda

Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
                                             Subject  \
0                                                      
1             New login to your Hugging Face account   
2  [Update] Changes to the Google Cloud Third-Par...   
3                                Alerta de seguridad   
4             ¿Conviene o no aceptar un contrato de…   

                                              From  \
0              Camilo Soto <csoto981217@gmail.com>   
1             huggingface <website@huggingface.co>   
2  Google Cloud <CloudPlatform-noreply@google.com>   
3            Google <no-reply@accounts.google.com>   
4    LinkedIn <notifications-noreply@linkedin.com>   

                                             Mensaje  
0                         Me gusta mucho la vida\r\n  
1  \r\n<html>\r\n<head>\r\n\r\n<style>\r\nhtml {\...  
2  Dear Camilo,\r\n\r\nWe are writing 

In [20]:
df

,Subject,From,Mensaje
0,,Camilo Soto <csoto981217@gmail.com>,me gusta mucho la vida
1,New login to your Hugging Face account,huggingface <website@huggingface.co>,html head style html fontfamily serif fontsize...
2,[Update] Changes to the Google Cloud Third-Par...,Google Cloud <CloudPlatform-noreply@google.com>,dear camilo we are writing to let you know abo...
3,Alerta de seguridad,Google <no-reply@accounts.google.com>,image google se creó una contraseña de la apli...
4,¿Conviene o no aceptar un contrato de…,LinkedIn <notifications-noreply@linkedin.com>,conviene o no aceptar un contrato de trabajo t...


In [21]:
import pandas as pd
from transformers import pipeline
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from getpass import getpass

# Supongamos que tienes un DataFrame con una columna llamada "mensaje"


pipe = pipeline("sentiment-analysis")

# Función para truncar mensajes largos
def truncar_mensaje(mensaje, max_length=512):
    tokens = pipe.tokenizer.encode(mensaje, add_special_tokens=True)
    if len(tokens) > max_length:
        tokens = tokens[:max_length-1] + [pipe.tokenizer.sep_token_id]
    return pipe.tokenizer.decode(tokens, skip_special_tokens=True)

# Función para aplicar el modelo y obtener la predicción
def obtener_prediccion(mensaje):
    mensaje_truncado = truncar_mensaje(mensaje)
    resultado = pipe(mensaje_truncado)[0]  # Obtenemos el primer resultado
    return resultado['label'], resultado['score']  # Devolvemos la etiqueta y la puntuación

# Función para enviar un correo electrónico
def enviar_correo(subject, body, to_email, from_email, password):
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(from_email, password)
        text = msg.as_string()
        server.sendmail(from_email, to_email, text)
        server.quit()
        print("Correo enviado exitosamente a", to_email)
    except Exception as e:
        print(f"Error al enviar el correo a {to_email}: {e}")

# Credenciales de correo electrónico
from_email = username
password = password
negative_email = "csoto981217@ejemplo.com"
positive_email = "csoto981217@ejemplo.com"

# Aplicar la función a cada mensaje en el DataFrame
def analizar_y_notificar(row):
    label, score = obtener_prediccion(row['Mensaje'])
    if label == 'NEGATIVE':
        subject = "Alerta de Mensaje negativo"
        body = f"Se ha detectado un Mensaje negativo:\n\n{row['Mensaje']}\n\nConfianza: {score:.2f}"
        enviar_correo(subject, body, negative_email, from_email, password)
    elif label == 'POSITIVE':
        subject = "Alerta de mensaje positivo"
        body = f"Se ha detectado un Mensaje positivo:\n\n{row['Mensaje']}\n\nConfianza: {score:.2f}"
        enviar_correo(subject, body, positive_email, from_email, password)
    return label, score

df[['prediccion', 'confianza']] = df.apply(analizar_y_notificar, axis=1, result_type="expand")

# Mostrar el DataFrame con las predicciones
print(df)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Correo enviado exitosamente a csoto981217@ejemplo.com
Correo enviado exitosamente a csoto981217@ejemplo.com
Correo enviado exitosamente a csoto981217@ejemplo.com


Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


Correo enviado exitosamente a csoto981217@ejemplo.com
Correo enviado exitosamente a csoto981217@ejemplo.com
                                             Subject  \
0                                                      
1             New login to your Hugging Face account   
2  [Update] Changes to the Google Cloud Third-Par...   
3                                Alerta de seguridad   
4             ¿Conviene o no aceptar un contrato de…   

                                              From  \
0              Camilo Soto <csoto981217@gmail.com>   
1             huggingface <website@huggingface.co>   
2  Google Cloud <CloudPlatform-noreply@google.com>   
3            Google <no-reply@accounts.google.com>   
4    LinkedIn <notifications-noreply@linkedin.com>   

                                             Mensaje prediccion  confianza  
0                             me gusta mucho la vida   POSITIVE   0.998631  
1  html head style html fontfamily serif fontsize...   NEGATIVE   0.997491  

In [22]:
df

,Subject,From,Mensaje,prediccion,confianza
0,,Camilo Soto <csoto981217@gmail.com>,me gusta mucho la vida,POSITIVE,0.998631
1,New login to your Hugging Face account,huggingface <website@huggingface.co>,html head style html fontfamily serif fontsize...,NEGATIVE,0.997491
2,[Update] Changes to the Google Cloud Third-Par...,Google Cloud <CloudPlatform-noreply@google.com>,dear camilo we are writing to let you know abo...,NEGATIVE,0.980753
3,Alerta de seguridad,Google <no-reply@accounts.google.com>,image google se creó una contraseña de la apli...,NEGATIVE,0.982408
4,¿Conviene o no aceptar un contrato de…,LinkedIn <notifications-noreply@linkedin.com>,conviene o no aceptar un contrato de trabajo t...,NEGATIVE,0.995482
